# Examples with JSON Log Entries

> **These are Go notebooks**: In order to use the GoNB Jupyter Kernel, please install GoNB from here: [https://github.com/janpfeifer/gonb](https://github.com/janpfeifer/gonb)

Note also that for local package development, you can put: `!*go mod edit -replace "github.com/umbralcalc/stochadex=/path/to/stochadex"` at the top of any cell.

## Saving logs (standard)

When running larger or more complex simulations it sometimes makes sense to simply log the generated data in some file for later analysis. We demonstrate this functionality in the example below by setting up a standard logging output function.

In [2]:
import "github.com/umbralcalc/stochadex/pkg/simulator"

%%

// Manually setup a simulation config
generator := simulator.NewConfigGenerator()

// Manually configure a simulation state partition
generator.SetPartition(&simulator.PartitionConfig{
	Name:              "first_wiener_process",
	Iteration:         &continuous.WienerProcessIteration{},
	Params:            simulator.NewParams(map[string][]float64{
		"variances": {1.0, 2.0, 3.0, 4.0},
	}),
	InitStateValues:   []float64{0.0, 0.0, 0.0, 0.0},
	StateHistoryDepth: 1,
	Seed:              12345,
})

// Manually configure another simulation state partition
generator.SetPartition(&simulator.PartitionConfig{
	Name:              "second_wiener_process",
	Iteration:         &continuous.WienerProcessIteration{},
	Params:            simulator.NewParams(map[string][]float64{
		"variances": {1.0, 2.0},
	}),
	InitStateValues:   []float64{0.0, 0.0},
	StateHistoryDepth: 1,
	Seed:              5678,
})

// Manually configure the extra simulation run specs
generator.SetSimulation(&simulator.SimulationConfig{
	OutputCondition: &simulator.EveryStepOutputCondition{},
	OutputFunction: simulator.NewJsonLogOutputFunction("./data/test.log"),
	TerminationCondition: &simulator.NumberOfStepsTerminationCondition{
		MaxNumberOfSteps: 100,
	},
	TimestepFunction: &simulator.ConstantTimestepFunction{
		Stepsize: 1.0,
	},
	InitTimeValue: 0.0,
})

// Setup a simulation with this configuration
coordinator := simulator.NewPartitionCoordinator(generator.GenerateConfigs())

// Run the simulation
coordinator.Run()

## Saving logs (advanced)

The simple logging output function must apply a global synchronisation lock when writing each log entry to the file, which can lead to undesirable performance bottlenecks in some situations. To alleviate this issue, we can instead create a new thread (goroutine) and logging channel like in the updated example below.

In [3]:
import "github.com/umbralcalc/stochadex/pkg/simulator"

%%

// Manually setup a simulation config
generator := simulator.NewConfigGenerator()

// Manually configure a simulation state partition
generator.SetPartition(&simulator.PartitionConfig{
	Name:              "first_wiener_process",
	Iteration:         &continuous.WienerProcessIteration{},
	Params:            simulator.NewParams(map[string][]float64{
		"variances": {1.0, 2.0, 3.0, 4.0},
	}),
	InitStateValues:   []float64{0.0, 0.0, 0.0, 0.0},
	StateHistoryDepth: 1,
	Seed:              12345,
})

// Manually configure another simulation state partition
generator.SetPartition(&simulator.PartitionConfig{
	Name:              "second_wiener_process",
	Iteration:         &continuous.WienerProcessIteration{},
	Params:            simulator.NewParams(map[string][]float64{
		"variances": {1.0, 2.0},
	}),
	InitStateValues:   []float64{0.0, 0.0},
	StateHistoryDepth: 1,
	Seed:              5678,
})

// Create a higher performance logging channel
logChannel := simulator.NewJsonLogChannelOutputFunction("./data/test.log")
defer logChannel.Close()

// Manually configure the extra simulation run specs
generator.SetSimulation(&simulator.SimulationConfig{
	OutputCondition: &simulator.EveryStepOutputCondition{},
	OutputFunction: logChannel,
	TerminationCondition: &simulator.NumberOfStepsTerminationCondition{
		MaxNumberOfSteps: 100,
	},
	TimestepFunction: &simulator.ConstantTimestepFunction{
		Stepsize: 1.0,
	},
	InitTimeValue: 0.0,
})

// Setup a simulation with this configuration
coordinator := simulator.NewPartitionCoordinator(generator.GenerateConfigs())

// Run the simulation
coordinator.Run()

## Loading logs

Having already generated the JSON log entries from some simulation run, we can easily load these logs back into memory. We do this in the example below, which uses the logs generated by the cells above.

In [ ]:
import (
	"github.com/umbralcalc/stochadex/pkg/analysis"

	"github.com/go-echarts/go-echarts/v2/opts"
	gonb_echarts "github.com/janpfeifer/gonb-echarts"
)

%%

// Create a simulator.StateTimeStorage from a log entries file
storage, _ := analysis.NewStateTimeStorageFromJsonLogEntries("./data/test.log")

// Reference the plotting data for the x-axis
xRef := analysis.DataRef{Plotting: &analysis.DataPlotting{IsTime: true}}

// Reference the plotting data for the y-axis
yRefs := []analysis.DataRef{{PartitionName: "first_wiener_process"}}

// Create a scatter plot from partitions in a simulator.StateTimeStorage
scatter := analysis.NewScatterPlotFromPartition(storage, xRef, yRefs)

// Display the plot in a Go notebook
gonb_echarts.Display(scatter, "width: 1024px; height:400px; background: white;")